# Projet INSEE - MOBILITÉ TRANSFRONTALIÈRE

# Modèle 1 : probit simple 

## Préparation de la base donnée

In [1]:
import pandas as pd

# Charger le fichier X
X = pd.read_csv('/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/ML_READY/FINAL_AFTER_ARBITRAGES/X_final_clean.csv')

print("="*80)
print("STRUCTURE DU FICHIER X_final_clean.csv")
print("="*80)

print(f"\nDimensions : {X.shape[0]:,} lignes × {X.shape[1]} colonnes")

print("\n" + "="*80)
print("LISTE DES COLONNES")
print("="*80)
for i, col in enumerate(X.columns):
    print(f"  {i+1:>2}. {col}")

print("\n" + "="*80)
print("TYPES DE DONNÉES")
print("="*80)
print(X.dtypes)

print("\n" + "="*80)
print("APERÇU (5 premières lignes)")
print("="*80)
print(X.head())

print("\n" + "="*80)
print("STATISTIQUES")
print("="*80)
print(X.describe().T)

print("\n" + "="*80)
print("VALEURS MANQUANTES")
print("="*80)
print(X.isnull().sum()[X.isnull().sum() > 0])

STRUCTURE DU FICHIER X_final_clean.csv

Dimensions : 494,483 lignes × 81 colonnes

LISTE DES COLONNES
   1. AGEREV
   2. AGEREV_sq
   3. GS_1
   4. GS_2
   5. GS_3
   6. GS_4
   7. GS_5
   8. GS_6
   9. EMPL_11
  10. EMPL_12
  11. EMPL_13
  12. EMPL_14
  13. EMPL_15
  14. EMPL_16
  15. EMPL_21
  16. EMPL_22
  17. EMPL_23
  18. INATC_1
  19. INATC_2
  20. COUPLE_1
  21. COUPLE_2
  22. NENFR_0
  23. NENFR_1
  24. NENFR_2
  25. NENFR_3
  26. NENFR_4
  27. NENFR_Z
  28. NA5_AZ
  29. NA5_BE
  30. NA5_FZ
  31. NA5_GU
  32. NA5_OQ
  33. DIPL_1
  34. DIPL_11
  35. DIPL_12
  36. DIPL_13
  37. DIPL_14
  38. DIPL_15
  39. DIPL_16
  40. DIPL_17
  41. DIPL_18
  42. DIPL_19
  43. DIPL_2
  44. DIPL_3
  45. ETUD_1
  46. ETUD_2
  47. SANI_0
  48. SANI_1
  49. SANI_2
  50. SANI_X
  51. SEXE_1
  52. SEXE_2
  53. TP_1
  54. TP_2
  55. DEPT_10
  56. DEPT_51
  57. DEPT_52
  58. DEPT_54
  59. DEPT_55
  60. DEPT_57
  61. DEPT_67
  62. DEPT_68
  63. DEPT_88
  64. STOCD_10.0
  65. STOCD_21.0
  66. STOCD_22.0
  

# Choix des catégories de références + check vif

In [2]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor

# ============================================
# CHARGER DONNÉES
# ============================================
X = pd.read_csv('/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/ML_READY/FINAL_AFTER_ARBITRAGES/X_final_clean.csv')
y = pd.read_csv('/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/ML_READY/FINAL_AFTER_ARBITRAGES/y_final.csv')

df = X.copy()
df['Y'] = y.values.ravel() if len(y.shape) > 1 else y.values

print("="*80)
print("DIAGNOSTIC PRÉ-PROBIT : 17 VARIABLES CONCEPTUELLES")
print("="*80)
print(f"\nBase : {len(df):,} obs | Transfrontaliers : {df['Y'].sum():,} ({df['Y'].mean()*100:.2f}%)")

# ============================================
# 1. MAPPING DUMMIES → VARIABLES CONCEPTUELLES
# ============================================
print("\n" + "="*80)
print("1. STRUCTURE : 81 DUMMIES → 17 VARIABLES CONCEPTUELLES")
print("="*80)

# Identifier les préfixes (variables conceptuelles)
def get_prefix(col):
    if col in ['AGEREV', 'AGEREV_sq']:
        return 'AGEREV'
    elif col.startswith('DNAI_'):
        return 'DNAI'
    elif '_' in col:
        return col.rsplit('_', 1)[0]
    else:
        return col

# Regrouper colonnes par variable conceptuelle
var_mapping = {}
for col in X.columns:
    prefix = get_prefix(col)
    if prefix not in var_mapping:
        var_mapping[prefix] = []
    var_mapping[prefix].append(col)

print(f"\n{'Variable conceptuelle':<15} {'Nb dummies':>12} {'Colonnes'}")
print("-"*80)
for var, cols in sorted(var_mapping.items()):
    cols_str = ', '.join(cols[:3])
    if len(cols) > 3:
        cols_str += f" ... (+{len(cols)-3})"
    print(f"{var:<15} {len(cols):>12} {cols_str}")

# ============================================
# 2. DÉTAIL PAR VARIABLE : EFFECTIFS + TAUX TRANSFRONTALIER
# ============================================
print("\n" + "="*80)
print("2. MODALITÉS PAR VARIABLE (effectifs + taux transfrontalier)")
print("="*80)
print("→ Pour choisir la CATÉGORIE DE RÉFÉRENCE du Probit\n")

ref_suggestions = {}

for var, cols in sorted(var_mapping.items()):
    print(f"\n{'─'*60}")
    print(f"📊 {var} ({len(cols)} modalités)")
    print(f"{'─'*60}")
    
    if var == 'AGEREV':
        # Variable continue
        print(f"  Type: CONTINUE")
        print(f"  Moyenne: {X['AGEREV'].mean():.1f} ans | Écart-type: {X['AGEREV'].std():.1f}")
        print(f"  Min: {X['AGEREV'].min()} | Max: {X['AGEREV'].max()}")
        print(f"  → Pas de catégorie de référence (variable continue)")
        ref_suggestions[var] = None
        continue
    
    print(f"  {'Modalité':<20} {'Effectif':>12} {'%':>8} {'Taux transf.':>14} {'Suggestion'}")
    print(f"  {'-'*70}")
    
    modal_stats = []
    for col in cols:
        modalite = col.replace(var + '_', '')
        effectif = X[col].sum()
        pct = effectif / len(X) * 100
        taux_transf = df[df[col] == 1]['Y'].mean() * 100 if effectif > 0 else 0
        modal_stats.append({
            'col': col,
            'modalite': modalite,
            'effectif': effectif,
            'pct': pct,
            'taux': taux_transf
        })
    
    # Trier par effectif décroissant
    modal_stats = sorted(modal_stats, key=lambda x: x['effectif'], reverse=True)
    
    # La plus fréquente = suggestion de référence
    ref_col = modal_stats[0]['col']
    ref_suggestions[var] = ref_col
    
    for i, m in enumerate(modal_stats):
        suggestion = "← REF (plus fréquente)" if i == 0 else ""
        print(f"  {m['modalite']:<20} {m['effectif']:>12,} {m['pct']:>7.1f}% {m['taux']:>13.1f}% {suggestion}")

# ============================================
# 3. VIF SUR VARIABLES CONCEPTUELLES
# ============================================
print("\n" + "="*80)
print("3. VIF - MULTICOLINÉARITÉ (sur 17 variables conceptuelles)")
print("="*80)
print("Méthode: Label encoding des catégorielles → VIF")
print("Seuils : VIF < 5 ✓ | 5-10 ⚠️ | ≥10 🔴\n")

# Reconstruire les variables conceptuelles en label encoding
X_conceptual = pd.DataFrame()

for var, cols in var_mapping.items():
    if var == 'AGEREV':
        X_conceptual['AGEREV'] = X['AGEREV']
        # On n'inclut pas AGEREV_sq car colinéaire par construction
    else:
        # Reconstruire la variable catégorielle
        # Trouver quelle modalité = 1 pour chaque ligne
        temp = X[cols].idxmax(axis=1)
        # Encoder en numérique
        X_conceptual[var] = pd.factorize(temp)[0]

# Calculer VIF
print(f"Calcul VIF sur {len(X_conceptual.columns)} variables...")

vif_results = []
for i, col in enumerate(X_conceptual.columns):
    try:
        vif = variance_inflation_factor(X_conceptual.values.astype(float), i)
        vif_results.append({'Variable': col, 'VIF': vif})
    except Exception as e:
        vif_results.append({'Variable': col, 'VIF': np.nan})

vif_df = pd.DataFrame(vif_results).sort_values('VIF', ascending=False)

print(f"\n{'Variable':<15} {'VIF':>10} {'Statut'}")
print("-"*40)
for _, row in vif_df.iterrows():
    vif = row['VIF']
    if pd.isna(vif) or np.isinf(vif):
        statut = "❌ Erreur"
        vif_str = "∞"
    elif vif >= 10:
        statut = "🔴 Sévère"
        vif_str = f"{vif:.2f}"
    elif vif >= 5:
        statut = "⚠️ Modéré"
        vif_str = f"{vif:.2f}"
    else:
        statut = "✓ OK"
        vif_str = f"{vif:.2f}"
    print(f"{row['Variable']:<15} {vif_str:>10} {statut}")

vif_clean = vif_df[~vif_df['VIF'].isna() & ~np.isinf(vif_df['VIF'])]['VIF']
print("-"*40)
print(f"VIF moyen : {vif_clean.mean():.2f} | VIF max : {vif_clean.max():.2f}")

if vif_clean.max() < 5:
    print("\n✅ PAS DE MULTICOLINÉARITÉ PROBLÉMATIQUE")
else:
    print(f"\n⚠️ {(vif_clean >= 5).sum()} variable(s) avec VIF ≥ 5")

# ============================================
# 4. CORRÉLATIONS FORTES ENTRE VARIABLES CONCEPTUELLES
# ============================================
print("\n" + "="*80)
print("4. CORRÉLATIONS FORTES (|ρ| > 0.4)")
print("="*80)

corr = X_conceptual.corr(method='spearman')

pairs = []
for i in range(len(corr.columns)):
    for j in range(i+1, len(corr.columns)):
        r = corr.iloc[i, j]
        if abs(r) > 0.4:
            pairs.append((corr.columns[i], corr.columns[j], r))

if pairs:
    print(f"\n{'Variable 1':<15} {'Variable 2':<15} {'ρ Spearman':>12}")
    print("-"*45)
    for v1, v2, r in sorted(pairs, key=lambda x: abs(x[2]), reverse=True):
        print(f"{v1:<15} {v2:<15} {r:>12.3f}")
else:
    print("\n✅ Aucune corrélation |ρ| > 0.4")

# ============================================
# 5. RÉCAPITULATIF CATÉGORIES DE RÉFÉRENCE
# ============================================
print("\n" + "="*80)
print("5. CATÉGORIES DE RÉFÉRENCE SUGGÉRÉES (pour Probit)")
print("="*80)
print("Critère: modalité la plus fréquente (convention standard)\n")

print(f"{'Variable':<15} {'Référence suggérée':<30} {'Justification'}")
print("-"*70)

# Labels lisibles
labels = {
    'GS_5': 'Employés',
    'EMPL_16': 'CDI/Fonctionnaire',
    'INATC_1': 'Français',
    'COUPLE_1': 'En couple',
    'NENFR_0': '0 enfant',
    'NA5_GU': 'Commerce/Services',
    'DIPL_13': 'CAP/BEP',
    'ETUD_2': 'Non étudiant',
    'SANI_2': 'Salle de bain',
    'SEXE_1': 'Homme',
    'TP_1': 'Temps complet',
    'DEPT_57': 'Moselle',
    'STOCD_10.0': 'Propriétaire',
    'VOIT_2.0': '2 voitures',
    'TYPL_1.0': 'Maison',
    'DNAI_NEGRANDEST': 'Né Grand Est'
}

for var, ref_col in ref_suggestions.items():
    if ref_col is None:
        print(f"{var:<15} {'(continue)':<30} {'Pas de référence'}")
    else:
        label = labels.get(ref_col, ref_col.replace(var + '_', ''))
        effectif = X[ref_col].sum()
        pct = effectif / len(X) * 100
        print(f"{var:<15} {label:<30} {pct:.1f}% des obs")

print("\n" + "="*80)
print("FIN DU DIAGNOSTIC")
print("="*80)

DIAGNOSTIC PRÉ-PROBIT : 17 VARIABLES CONCEPTUELLES

Base : 494,483 obs | Transfrontaliers : 44,264 (8.95%)

1. STRUCTURE : 81 DUMMIES → 17 VARIABLES CONCEPTUELLES

Variable conceptuelle   Nb dummies Colonnes
--------------------------------------------------------------------------------
AGEREV                     2 AGEREV, AGEREV_sq
COUPLE                     2 COUPLE_1, COUPLE_2
DEPT                       9 DEPT_10, DEPT_51, DEPT_52 ... (+6)
DIPL                      12 DIPL_1, DIPL_11, DIPL_12 ... (+9)
DNAI                       3 DNAI_NEAUTREFR, DNAI_NEETRANGER, DNAI_NEGRANDEST
EMPL                       9 EMPL_11, EMPL_12, EMPL_13 ... (+6)
ETUD                       2 ETUD_1, ETUD_2
GS                         6 GS_1, GS_2, GS_3 ... (+3)
INATC                      2 INATC_1, INATC_2
NA5                        5 NA5_AZ, NA5_BE, NA5_FZ ... (+2)
NENFR                      6 NENFR_0, NENFR_1, NENFR_2 ... (+3)
SANI                       4 SANI_0, SANI_1, SANI_2 ... (+1)
SEXE            

# SYNTHÈSE EXHAUSTIVE DES VARIABLES DU MODÈLE PROBIT
## Projet INSEE - Mobilité Transfrontalière Grand Est

**Base de données** : 494,483 observations | Transfrontaliers : 44,264 (8.95%)

---

# 1. AGEREV — Âge en années révolues

**Source** : INDCVI (Fichier Individus)

| Propriété             | Valeur |
|-----------            |--------|
| **Type**              | Continue |
| **Description INSEE** | Âge en années révolues détaillé (âge au dernier anniversaire) |
| **Plage**             | 0 à 120 ans |
| **Dans le modèle**    | 15-101 ans (actifs occupés uniquement) |

**Statistiques dans la BDD** :
- Moyenne : 41.5 ans
- Écart-type : 12.5 ans
- Min : 15 ans | Max : 101 ans

**Spécification probit** : 
- `AGEREV` (effet linéaire)
- `AGEREV_sq` (effet quadratique) → Effet en U inversé confirmé par Elastic Net

**Pas de catégorie de référence** (variable continue)

---

# 2. COUPLE — Déclaration de vie en couple

**Source** : INDCVI (Fichier Individus)

| Code  | Libellé INSEE                    | Effectif | % Base | Taux transf. |
|------ |---------------                   |----------|--------|--------------|
| **1** | A déclaré vivre en couple        | 325,880  | 65.9%  | 9.7% |
| 2     | A déclaré ne pas vivre en couple | 168,603  | 34.1%  | 7.5% |

**Catégorie de référence suggérée** : `COUPLE_1` (En couple) — 65.9% des observations

**Note** : Variable déclarative, pas de distinction marié/pacsé/concubinage ici (voir STAT_CONJ si nécessaire)

---

# 3. DEPT — Département du lieu de résidence

**Source** : INDCVI + MOBPRO

| Code   | Département        | Effectif | % Base | Taux transf. | Interprétation |
|------  |-------------       |----------|--------|--------------|----------------|
| **67** | Bas-Rhin           | 118,398  | 23.9%  | 4.6% | Strasbourg, éloigné frontière |
| 57     | Moselle            | 98,869   | 20.0%  | **21.6%** | Proximité Luxembourg |
| 68     | Haut-Rhin          | 73,587   | 14.9%  | **14.1%** | Proximité Suisse/Allemagne|
| 54     | Meurthe-et-Moselle | 64,924   | 13.1%  | 9.7% | Proximité Luxembourg |
| 51     | Marne              | 53,239   | 10.8%  | 0.1% | Éloigné des frontières |
| 88     | Vosges             | 29,098   | 5.9%   | 0.2% | Relief montagneux |
| 10     | Aube               | 27,234   | 5.5%   | 0.1% | Éloigné des frontières |
| 55     | Meuse              | 14,885   | 3.0%   | 4.7% | Entre Metz et frontière |
| 52     | Haute-Marne        | 14,249   | 2.9%   | 0.1% | Éloigné des frontières |

**Catégorie de référence suggérée** : `DEPT_67` (Bas-Rhin) — 23.9% des observations

**Attention** : Le diagnostic suggère DEPT_67 (plus fréquent), mais économétriquement on pourrait préférer un département "neutre" éloigné des frontières (ex: DEPT_51 Marne) pour que tous les coefficients des départements frontaliers soient positifs et comparables.

## Choix final : alternative selectionnée pour DEPT
Utiliser DEPT_51 (Marne) comme référence pour avoir tous les coefficients des départements frontaliers en positif, ce qui facilite l'interprétation.

---

# 4. DIPL — Diplôme le plus élevé

**Source** : INDCVI (Fichier Individus)

| Code | Libellé INSEE complet                       | Effectif | % Base | Taux transf. |
|------|----------------------                       |----------|--------|--------------|
| 01 | Pas de scolarité ou arrêt avant fin primaire        | 3,431  | 0.7%  | 5.4% |
| 02 | Aucun diplôme, scolarité fin primaire/avant collège | 10,348 | 2.1%  | 8.1% |
| 03 | Aucun diplôme, scolarité jusqu'à fin collège ou +   | 26,638 | 5.4%  | 9.1% |
| 11 | CEP (certificat d'études primaires)                 | 4,062  | 0.8%  | 12.0% |
| 12 | BEPC, brevet élémentaire, DNB                       | 16,298 | 3.3%  | 8.1% |
| **13** | **CAP, BEP ou diplôme équivalent**              | 117,859|**23.8%**| 9.6% |
| 14 | Bac général/techno, brevet supérieur, DAEU          | 50,473 | 10.2% | 7.8% |
| 15 | Bac pro, brevet professionnel/technicien            | 56,154 | 11.4% | 8.7% |
| 16 | BTS, DUT, Deug, Deust (Bac+2)                       | 75,404 | 15.2% | 7.9% |
| 17 | Licence, licence pro, maîtrise (Bac+3/4)            | 63,260 | 12.8% | 8.3% |
| 18 | Master, DEA, DESS, grande école (Bac+5)             | 65,128 | 13.2% | **10.7%** |
| 19 | Doctorat de recherche (hors santé)                  | 5,428  | 1.1%  | **11.2%** |

**Catégorie de référence suggérée** : `DIPL_13` (CAP/BEP) — 23.8% des observations

**Interprétation** : Les très diplômés (18, 19) et les CEP (11) ont les taux transfrontaliers les plus élevés. L'effet diplôme n'est pas linéaire.

---

# 5. DNAI — Département de naissance (recodé)

**Source** : INDCVI — Variable DNAI recodée en 3 modalités

| Code recodé    | Description                        | Effectif | % Base | Taux transf. |
|-------------   |-------------                       |----------|--------|--------------|
| **NEGRANDEST** | Né dans le Grand Est            | 360,009 | 72.8% | 8.1% |
| NEAUTREFR      | Né ailleurs en France           | 71,270 | 14.4% | 5.8% |
| NEETRANGER     | Né à l'étranger (code 99 INSEE) | 63,204 | 12.8% | **17.5%** |

**Variable INSEE originale** : 
- Codes 08, 10, 51, 52, 54, 55, 57, 67, 68, 88 → NEGRANDEST
- Autres codes français → NEAUTREFR
- Code 99 → NEETRANGER

**Catégorie de référence suggérée** : `DNAI_NEGRANDEST` — 72.8% des observations

**Interprétation économique** : Les personnes nées à l'étranger ont 2× plus de chances d'être transfrontaliers (réseaux, expérience internationale, moins d'ancrage territorial).

---

# 6. EMPL — Condition d'emploi

**Source** : INDCVI + MOBPRO

| Code | Libellé INSEE                                 | Effectif | % Base | Taux transf. |
|------|---------------                                |----------|--------|--------------|
| 11 | Contrat d'apprentissage ou professionnalisation  | 14,172  | 2.9%    | 1.8% |
| 12 | Placés par agence d'intérim                      | 10,308  | 2.1%    | **10.8%** |
| 13 | Emplois aidés (CUI, avenir, etc.)                | 2,437   | 0.5%    | 1.3% |
| 14 | Stagiaires rémunérés en entreprise               | 1,350   | 0.3%    | 9.3% |
| 15 | Autres emplois à durée limitée (CDD, saisonnier) | 40,025  | 8.1%    | 5.7% |
| **16** | **CDI, titulaire fonction publique**         | 374,974 |**75.8%**| **10.4%** |
| 21 | Non salariés : Indépendants                      | 29,943  | 6.1%    | 2.6% |
| 22 | Non salariés : Employeurs                        | 20,770  | 4.2%    | 3.6% |
| 23 | Non salariés : Aides familiaux                   | 504     | 0.1%    | 3.8% |

**Catégorie de référence suggérée** : `EMPL_16` (CDI/Fonctionnaire) — 75.8% des observations

**Interprétation** : 
- Les CDI ont le taux transfrontalier le plus élevé (stabilité → acceptation déplacements)
- L'intérim (12) aussi élevé car souvent transfrontalier (agences spécialisées Luxembourg)
- Les indépendants (21) très faibles car activité ancrée localement

---

# 7. ETUD — Inscription dans un établissement d'enseignement

**Source** : INDCVI (Fichier Individus)

| Code  | Libellé INSEE         | Effectif | % Base | Taux transf. |
|------ |---------------        |----------|--------|--------------|
| 1     | Oui (inscrit)         | 25,247   | 5.1%   | 3.1% |
| **2** | **Non (non inscrit)** | 469,236  |**94.9%**| 9.3% |

**Catégorie de référence suggérée** : `ETUD_2` (Non étudiant) — 94.9% des observations

**Note** : Dans ta base, tu as filtré sur les actifs occupés, donc les "étudiants" ici sont des travailleurs-étudiants (alternance, formation continue).

---

# 8. GS — Groupe socioprofessionnel en 6 postes

**Source** : INDCVI + MOBPRO

| Code | Libellé INSEE                               | Effectif | % Base | Taux transf. |
|------|---------------                              |----------|--------|--------------|
| 1    | Agriculteurs exploitants                    | 5,990   | 1.2%    | **0.4%** |
| 2    | Artisans, commerçants, chefs d'entreprise   | 28,006  | 5.7%    | 3.2% |
| 3    | Cadres et professions intellect. supérieures| 80,429  | 16.3%   | 9.8% |
| 4    | Professions intermédiaires                  | 131,258 | 26.5%   | 8.2% |
|**5** | **Employés**                                | 133,175 |**26.9%**| 7.8% |
| 6    | Ouvriers                                    | 115,625 | 23.4%   | **12.3%** |

**Catégorie de référence suggérée** : `GS_5` (Employés) — 26.9% des observations

**Interprétation économique** :
- **Ouvriers (6)** : Taux le plus élevé (12.3%) → industrie transfrontalière (Luxembourg, Allemagne)
- **Agriculteurs (1)** : Taux quasi nul → activité terrienne par nature
- **Cadres (3)** : Taux élevé → différentiel salarial attractif Luxembourg/Suisse

---

# 9. INATC — Indicateur de nationalité condensé

**Source** : INDCVI + MOBPRO

| Code  | Libellé INSEE | Effectif | % Base | Taux transf. |
|------ |---------------|----------|--------|--------------|
| **1** | **Français**  | 458,904  |**92.8%**| 8.0% |
| 2     | Étrangers     | 35,579   | 7.2%   | **21.8%** |

**Catégorie de référence suggérée** : `INATC_1` (Français) — 92.8% des observations

**Interprétation** : Les étrangers résidant dans le Grand Est ont 2.7× plus de chances d'être transfrontaliers (proximité culturelle/linguistique avec pays voisin, réseaux, moins de contraintes administratives perçues).

---

# 10. NA5 — Activité économique regroupée en 5 postes

**Source** : INDCVI + MOBPRO (Nomenclature d'Activités Française)

| Code | Libellé INSEE                                | Effectif | % Base | Taux transf. |
|------|---------------                               |----------|--------|--------------|
| AZ | Agriculture, sylviculture et pêche              | 11,077 | 2.2%     | **0.7%** |
| BE | Industrie manufacturière, extractives et autres | 73,852 | 14.9%    | **12.9%** |
| FZ | Construction                                    | 32,768 | 6.6%     | **12.0%** |
| **GU** | **Commerce, transports et services divers** | 222,988 |**45.1%**| 11.4% |
| OQ | Administration publique, enseignement, santé    | 153,798 | 31.1%   | 3.5% |

**Catégorie de référence suggérée** : `NA5_GU` (Commerce/Services) — 45.1% des observations

**Interprétation économique** :
- **OQ (Administration)** : Taux très faible (3.5%) → emplois publics français par nature
- **AZ (Agriculture)** : Quasi nul → activité terrienne
- **BE (Industrie)** et **FZ (Construction)** : Très élevés → secteurs transfrontaliers typiques

---

# 11. NENFR — Nombre d'enfants de la famille (regroupé)

**Source** : INDCVI (Fichier Individus)

| Code  | Libellé INSEE                 | Effectif | % Base    | Taux transf. |
|------ |---------------                |----------|--------   |--------------|
| 0     | 0 enfant (couple sans enfant) | 116,349  | 23.5%     | 9.3% |
| **1** | **1 enfant**                  | 124,300  | **25.1%** | 9.3% |
| 2     | 2 enfants                     | 107,206  | 21.7%     | 9.5% |
| 3     | 3 enfants                     | 33,423   | 6.8%      | 8.4% |
| 4     | 4 enfants ou plus             | 10,672   | 2.2%      | 8.4% |
| Z     | Personne hors famille ou hors logement ordinaire | 102,533 | 20.7% | 7.8% |

**Catégorie de référence suggérée** : `NENFR_1` (1 enfant) — 25.1% des observations

**Choix final** : `NENFR_0` (0 enfant) car plus interprétable économiquement.

**Note** : La modalité "Z" correspond aux personnes vivant seules ou hors famille.

---

# 12. SANI — Installations sanitaires (France métropolitaine)

**Source** : INDCVI (Fichier Individus)

| Code  | Libellé INSEE                                | Effectif | % Base | Taux transf. |
|------ |---------------                               |----------|--------|--------------|
| 0     | Ni baignoire, ni douche                          | 650     | 0.1%  | 7.7% |
| 1     | Baignoire ou douche hors pièce réservée          | 15,297  | 3.1%  | **11.2%** |
| **2** | **Salle(s) de bains (avec douche ou baignoire)** | 475,443 |**96.1%**| 8.9% |
| X     | Hors logement ordinaire                          | 3,093   | 0.6%  | 0.5% |

**Catégorie de référence suggérée** : `SANI_2` (Salle de bains standard) — 96.1% des observations

**Interprétation** : Variable proxy du niveau de confort/standing du logement. Peut capter des effets de richesse.

---

# 13. SEXE — Sexe

**Source** : INDCVI + MOBPRO

| Code   | Libellé INSEE | Effectif | % Base | Taux transf. |
|------ |--------------  |----------|--------|--------------|
| **1** | **Hommes**     | 256,784  | **51.9%** | **10.5%** |
| 2     | Femmes         | 237,699  | 48.1%      | 7.2% |

**Catégorie de référence suggérée** : `SEXE_1` (Homme) — 51.9% des observations

**Interprétation économique** : Les hommes ont 1.5× plus de chances d'être transfrontaliers. Effet à explorer avec interactions (SEXE × NENFR, SEXE × EMPL) dans le modèle 2.

---

# 14. STOCD — Statut d'occupation détaillé du logement

**Source** : INDCVI + MOBPRO

| Code   | Libellé INSEE                       | Effectif | % Base | Taux transf. |
|------  |---------------                      |--------- |--------|--------------|
| 00     | Logement ordinaire inoccupé         | — | — | — |
| **10** | **Propriétaire**                    | 302,583  |**61.2%**| **10.4%** |
| 21     | Locataire logement vide non HLM     | 111,524  | 22.6%   | 8.3% |
| 22     | Locataire logement vide HLM         | 59,162   | 12.0%   | 4.0% |
| 23     | Locataire meublé ou chambre d'hôtel | 11,395   | 2.3%    | 6.4% |
| 30     | Logé gratuitement                   | 9,819    | 2.0%    | 5.3% |

**Catégorie de référence suggérée** : `STOCD_10` (Propriétaire) — 61.2% des observations

**Note** : Dans ta base, le code apparaît comme `STOCD_10.0` (float) → à vérifier/nettoyer.

**Interprétation** : Les propriétaires ont le taux le plus élevé (revenus → mobilité acceptée). Les locataires HLM ont le taux le plus faible.

---

# 15. TP — Temps de travail

**Source** : INDCVI + MOBPRO

| Code | Libellé INSEE   | Effectif | % Base    | Taux transf. |
|------|---------------  |----------|--------   |--------------|
|**1** |**Temps complet**| 411,499  | **83.2%** | 9.1% |
| 2    | Temps partiel   | 82,984   | 16.8%     | 8.3% |

**Catégorie de référence suggérée** : `TP_1` (Temps complet) — 83.2% des observations

**Note** : La modalité "Z" (Sans objet) a été exclue car tu travailles sur les actifs occupés.

---

# 16. TYPL — Type de logement

**Source** : INDCVI + MOBPRO

| Code | Libellé INSEE                       | Effectif | % Base | Taux transf. |
|------|---------------                      |----------|--------|--------------|
| **1** | **Maison**                         | 293,723  |**59.4%**| **10.1%** |
| 2     | Appartement                        | 198,316  | 40.1%   | 7.4% |
| 3     | Logement-foyer                     | 1,543    | 0.3%    | 4.2% |
| 4     | Chambre d'hôtel                    | 288      | 0.1%    | 4.9% |
| 5     | Habitation de fortune              | 278      | 0.1%    | 6.8% |
| 6     | Pièce indépendante (entrée propre) | 335      | 0.1%    | 3.9% |

**Catégorie de référence suggérée** : `TYPL_1` (Maison) — 59.4% des observations

**Note** : Dans ta base, le code apparaît comme `TYPL_1.0` (float) → à vérifier/nettoyer.

**Interprétation** : Les habitants de maisons sont plus souvent transfrontaliers (périurbain/rural frontalier vs. centre-ville).

---

# 17. VOIT — Nombre de voitures du ménage

**Source** : INDCVI + MOBPRO

| Code | Libellé INSEE           | Effectif | % Base | Taux transf. |
|------|---------------          |----------|--------|--------------|
| 0     | Aucune voiture         | 33,957   | 6.9%   | 4.2% |
| 1     | Une seule voiture      | 176,994  | 35.8%  | 7.7% |
| **2** | **Deux voitures**      | 217,963  |**44.1%**| 10.1% |
| 3     | Trois voitures ou plus | 65,569   | 13.3%  | **11.2%** |

**Catégorie de référence suggérée** : `VOIT_2` (2 voitures) — 44.1% des observations

**Note** : Dans ta base, le code apparaît comme `VOIT_2.0` (float) → à vérifier/nettoyer.

**Interprétation économique** : Effet monotone croissant → plus de voitures = plus de mobilité possible = plus de travail transfrontalier. Effet revenu + effet accès.

---

# TABLEAU RÉCAPITULATIF DES CATÉGORIES DE RÉFÉRENCE

| Variable | Référence suggérée| Libellé           | % Base | Justification |
|----------|-------------------|---------          |--------|---------------|
| AGEREV   | —                 | Continue          | —     | Pas de référence |
| COUPLE   | `COUPLE_1`        | En couple         | 65.9% | Plus fréquente |
| DEPT     | `DEPT_51`         | Marne          | 10.8% |Choix arbitraire "effet frontière"
| DIPL     | `DIPL_13`         | CAP/BEP           | 23.8% | Plus fréquente |
| DNAI     | `DNAI_NEGRANDEST` | Né Grand Est      | 72.8% | Plus fréquente |
| EMPL     | `EMPL_16`         | CDI/Fonctionnaire | 75.8% | Plus fréquente |
| ETUD     | `ETUD_2`          | Non étudiant      | 94.9% | Plus fréquente |
| GS       | `GS_5`            | Employés          | 26.9% | Plus fréquente |
| INATC    | `INATC_1`         | Français          | 92.8% | Plus fréquente |
| NA5      | `NA5_GU`          | Commerce/Services | 45.1% | Plus fréquente |
| NENFR    | `NENFR_0`         | 0 enfant     | 23.5% |Choix arbitraire "effet parentalité"
| SANI     | `SANI_2`          | Salle de bains    | 96.1% | Plus fréquente |
| SEXE     | `SEXE_1`          | Homme             | 51.9% | Plus fréquente |
| STOCD    | `STOCD_10`        | Propriétaire      | 61.2% | Plus fréquente |
| TP       | `TP_1`            | Temps complet     | 83.2% | Plus fréquente |
| TYPL     | `TYPL_1`          | Maison            | 59.4% | Plus fréquente |
| VOIT     | `VOIT_2`          | 1 voitures        | 35.8% | Choix arbitraire "médiane" |

---

# POINTS D'ATTENTION

## Variable NENFR
La modalité "Z" (hors famille) mélange :
- Personnes vivant seules
- Personnes en colocation hors famille
- Personnes en communauté

À interpréter avec prudence.

In [7]:
"""
================================================================================
PRÉPARATION DE LA BASE PROBIT - BDD_PROBIT
================================================================================
Création d'une base prête pour l'estimation probit en supprimant les catégories
de référence pour chaque variable catégorielle (éviter la multicolinéarité parfaite).

Catégories de référence retenues (17 variables conceptuelles) :
--------------------------------------------------------------------------------
Variable    | Référence      | Justification
--------------------------------------------------------------------------------
AGEREV      | (continue)     | Variable continue + terme quadratique
GS          | GS_5           | Employés - modalité la plus fréquente (26.9%)
EMPL        | EMPL_16        | CDI/Titulaire - modalité la plus fréquente (75.8%)
INATC       | INATC_1        | Français - modalité la plus fréquente (92.8%)
COUPLE      | COUPLE_1       | En couple - modalité la plus fréquente (65.9%)
NENFR       | NENFR_0        | 0 enfant - CHOIX UTILISATEUR (interprétation parentalité)
NA5         | NA5_GU         | Commerce/Services - modalité la plus fréquente (45.1%)
DIPL        | DIPL_13        | CAP/BEP - modalité la plus fréquente (23.8%)
ETUD        | ETUD_2         | Non étudiant - modalité la plus fréquente (94.9%)
SANI        | SANI_2         | Salle de bain standard - modalité la plus fréquente (96.1%)
SEXE        | SEXE_1         | Homme - modalité la plus fréquente (51.9%)
TP          | TP_1           | Temps complet - modalité la plus fréquente (83.2%)
DEPT        | DEPT_51        | Marne - CHOIX UTILISATEUR (département intérieur, 
            |                | coefficients = effet relatif à un dept non-frontalier)
STOCD       | STOCD_10.0     | Propriétaire - modalité la plus fréquente (61.2%)
VOIT        | VOIT_1.0       | 1 voiture - CHOIX UTILISATEUR (référence médiane)
TYPL        | TYPL_1.0       | Maison - modalité la plus fréquente (59.4%)
DNAI        | DNAI_NEGRANDEST| Né Grand Est - modalité la plus fréquente (72.8%)
================================================================================
"""

import pandas as pd
import numpy as np

# ============================================================================
# CHARGEMENT DES DONNÉES
# ============================================================================
print("=" * 80)
print("PRÉPARATION DE LA BASE PROBIT")
print("=" * 80)

# Charger X_final_clean
X = pd.read_csv('/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/ML_READY/FINAL_AFTER_ARBITRAGES/X_final_clean.csv')
print(f"\n✓ Base chargée : {X.shape[0]:,} observations × {X.shape[1]} variables")

# ============================================================================
# DÉFINITION DES CATÉGORIES DE RÉFÉRENCE À SUPPRIMER
# ============================================================================
references_a_supprimer = {
    # Variable catégorielle : colonne de référence à supprimer
    'GS': 'GS_5',              # Employés (26.9%)
    'EMPL': 'EMPL_16',         # CDI/Titulaire (75.8%)
    'INATC': 'INATC_1',        # Français (92.8%)
    'COUPLE': 'COUPLE_1',      # En couple (65.9%)
    'NENFR': 'NENFR_0',        # 0 enfant - CHOIX UTILISATEUR
    'NA5': 'NA5_GU',           # Commerce/Services (45.1%)
    'DIPL': 'DIPL_13',         # CAP/BEP (23.8%)
    'ETUD': 'ETUD_2',          # Non étudiant (94.9%)
    'SANI': 'SANI_2',          # Salle de bain standard (96.1%)
    'SEXE': 'SEXE_1',          # Homme (51.9%)
    'TP': 'TP_1',              # Temps complet (83.2%)
    'DEPT': 'DEPT_51',         # Marne - CHOIX UTILISATEUR
    'STOCD': 'STOCD_10.0',     # Propriétaire (61.2%)
    'VOIT': 'VOIT_1.0',        # 1 voiture - CHOIX UTILISATEUR
    'TYPL': 'TYPL_1.0',        # Maison (59.4%)
    'DNAI': 'DNAI_NEGRANDEST', # Né Grand Est (72.8%)
}

# ============================================================================
# SUPPRESSION DES CATÉGORIES DE RÉFÉRENCE
# ============================================================================
print("\n" + "-" * 80)
print("CATÉGORIES DE RÉFÉRENCE SUPPRIMÉES")
print("-" * 80)

colonnes_a_supprimer = list(references_a_supprimer.values())
colonnes_presentes = [col for col in colonnes_a_supprimer if col in X.columns]
colonnes_absentes = [col for col in colonnes_a_supprimer if col not in X.columns]

if colonnes_absentes:
    print(f"\n⚠ Colonnes non trouvées : {colonnes_absentes}")

# Créer la base probit
bdd_probit = X.drop(columns=colonnes_presentes)

print(f"\n{'Variable':<12} {'Référence supprimée':<20} {'Justification'}")
print("-" * 70)
justifications = {
    'GS_5': 'Employés - plus fréquent (26.9%)',
    'EMPL_16': 'CDI/Titulaire - plus fréquent (75.8%)',
    'INATC_1': 'Français - plus fréquent (92.8%)',
    'COUPLE_1': 'En couple - plus fréquent (65.9%)',
    'NENFR_0': '0 enfant - CHOIX (interprétation parentalité)',
    'NA5_GU': 'Commerce/Services - plus fréquent (45.1%)',
    'DIPL_13': 'CAP/BEP - plus fréquent (23.8%)',
    'ETUD_2': 'Non étudiant - plus fréquent (94.9%)',
    'SANI_2': 'Salle de bain - plus fréquent (96.1%)',
    'SEXE_1': 'Homme - plus fréquent (51.9%)',
    'TP_1': 'Temps complet - plus fréquent (83.2%)',
    'DEPT_51': 'Marne - CHOIX (dept intérieur, non-frontalier)',
    'STOCD_10.0': 'Propriétaire - plus fréquent (61.2%)',
    'VOIT_1.0': '1 voiture - CHOIX (référence médiane)',
    'TYPL_1.0': 'Maison - plus fréquent (59.4%)',
    'DNAI_NEGRANDEST': 'Né Grand Est - plus fréquent (72.8%)',
}

for var, ref in references_a_supprimer.items():
    if ref in colonnes_presentes:
        print(f"{var:<12} {ref:<20} {justifications.get(ref, '')}")

# ============================================================================
# VÉRIFICATION DE LA STRUCTURE FINALE
# ============================================================================
print("\n" + "=" * 80)
print("STRUCTURE DE LA BASE PROBIT")
print("=" * 80)

print(f"\n✓ Dimensions finales : {bdd_probit.shape[0]:,} observations × {bdd_probit.shape[1]} variables")
print(f"  (suppression de {len(colonnes_presentes)} catégories de référence)")

# Compter les modalités par variable conceptuelle
print("\n" + "-" * 80)
print("RÉCAPITULATIF PAR VARIABLE CONCEPTUELLE")
print("-" * 80)
print(f"\n{'Variable':<12} {'Modalités gardées':<40} {'K-1'}")
print("-" * 70)

# Dictionnaire pour regrouper les colonnes par variable
variables_conceptuelles = {
    'AGEREV': ['AGEREV', 'AGEREV_sq'],
    'GS': [c for c in bdd_probit.columns if c.startswith('GS_')],
    'EMPL': [c for c in bdd_probit.columns if c.startswith('EMPL_')],
    'INATC': [c for c in bdd_probit.columns if c.startswith('INATC_')],
    'COUPLE': [c for c in bdd_probit.columns if c.startswith('COUPLE_')],
    'NENFR': [c for c in bdd_probit.columns if c.startswith('NENFR_')],
    'NA5': [c for c in bdd_probit.columns if c.startswith('NA5_')],
    'DIPL': [c for c in bdd_probit.columns if c.startswith('DIPL_')],
    'ETUD': [c for c in bdd_probit.columns if c.startswith('ETUD_')],
    'SANI': [c for c in bdd_probit.columns if c.startswith('SANI_')],
    'SEXE': [c for c in bdd_probit.columns if c.startswith('SEXE_')],
    'TP': [c for c in bdd_probit.columns if c.startswith('TP_')],
    'DEPT': [c for c in bdd_probit.columns if c.startswith('DEPT_')],
    'STOCD': [c for c in bdd_probit.columns if c.startswith('STOCD_')],
    'VOIT': [c for c in bdd_probit.columns if c.startswith('VOIT_')],
    'TYPL': [c for c in bdd_probit.columns if c.startswith('TYPL_')],
    'DNAI': [c for c in bdd_probit.columns if c.startswith('DNAI_')],
}

total_params = 0
for var, cols in variables_conceptuelles.items():
    if cols:
        n_cols = len(cols)
        total_params += n_cols
        cols_str = ', '.join(cols[:3])
        if len(cols) > 3:
            cols_str += f'... (+{len(cols)-3})'
        print(f"{var:<12} {cols_str:<40} {n_cols}")

print("-" * 70)
print(f"{'TOTAL':<12} {'':<40} {total_params} + constante")

# ============================================================================
# LISTE COMPLÈTE DES COLONNES
# ============================================================================
print("\n" + "-" * 80)
print("LISTE COMPLÈTE DES COLONNES (ordre alphabétique)")
print("-" * 80)

for i, col in enumerate(sorted(bdd_probit.columns), 1):
    print(f"  {i:2}. {col}")

# ============================================================================
# SAUVEGARDE
# ============================================================================
import os

out_dir = "/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/Probit_Ready"
os.makedirs(out_dir, exist_ok=True)

out_file = os.path.join(out_dir, "BDD_PROBIT.csv")   # choisis le nom que tu veux
bdd_probit.to_csv(out_file, index=False)


print("\n" + "=" * 80)
print("SAUVEGARDE")
print("=" * 80)
print(f"\n✓ Base sauvegardée : {output_path}")
print(f"  → {bdd_probit.shape[0]:,} observations")
print(f"  → {bdd_probit.shape[1]} variables explicatives")

# ============================================================================
# APERÇU FINAL
# ============================================================================
print("\n" + "-" * 80)
print("APERÇU (5 premières lignes)")
print("-" * 80)
print(bdd_probit.head())

# ============================================================================
# VÉRIFICATION : RANG DE LA MATRICE
# ============================================================================
print("\n" + "-" * 80)
print("VÉRIFICATION MULTICOLINÉARITÉ")
print("-" * 80)

# Vérifier que la matrice est de plein rang
rank = np.linalg.matrix_rank(bdd_probit.values)
print(f"\n✓ Rang de la matrice X : {rank}")
print(f"✓ Nombre de colonnes   : {bdd_probit.shape[1]}")

if rank == bdd_probit.shape[1]:
    print("\n✅ Matrice de plein rang - pas de multicolinéarité parfaite")
else:
    print(f"\n⚠ ATTENTION : Déficit de rang = {bdd_probit.shape[1] - rank}")
    print("  → Multicolinéarité parfaite détectée")

print("\n" + "=" * 80)
print("PRÉPARATION TERMINÉE")
print("=" * 80)

PRÉPARATION DE LA BASE PROBIT

✓ Base chargée : 494,483 observations × 81 variables

--------------------------------------------------------------------------------
CATÉGORIES DE RÉFÉRENCE SUPPRIMÉES
--------------------------------------------------------------------------------

Variable     Référence supprimée  Justification
----------------------------------------------------------------------
GS           GS_5                 Employés - plus fréquent (26.9%)
EMPL         EMPL_16              CDI/Titulaire - plus fréquent (75.8%)
INATC        INATC_1              Français - plus fréquent (92.8%)
COUPLE       COUPLE_1             En couple - plus fréquent (65.9%)
NENFR        NENFR_0              0 enfant - CHOIX (interprétation parentalité)
NA5          NA5_GU               Commerce/Services - plus fréquent (45.1%)
DIPL         DIPL_13              CAP/BEP - plus fréquent (23.8%)
ETUD         ETUD_2               Non étudiant - plus fréquent (94.9%)
SANI         SANI_2            

# Estimation économétrique en modèle probit binaire : DÉTERMINANTS DU TRAVAIL TRANSFRONTALIER

In [8]:
"""
================================================================================
MODÈLE 1 : PROBIT BINAIRE - DÉTERMINANTS DU TRAVAIL TRANSFRONTALIER
================================================================================
"""

import pandas as pd
import numpy as np
import statsmodels.api as sm

# ============================================================================
# CHARGEMENT
# ============================================================================
path = '/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/Probit_Ready/'
X = pd.read_csv(path + 'BDD_PROBIT.csv')
y = pd.read_csv(path + 'y_final.csv')['TRANSFRONTALIER']

# Ajouter constante
X_const = sm.add_constant(X)

print(f"Observations : {len(y):,}")
print(f"Transfrontaliers : {y.sum():,} ({y.mean()*100:.2f}%)")
print(f"Variables : {X_const.shape[1]} (dont constante)")

# ============================================================================
# ESTIMATION PROBIT
# ============================================================================
print("\nEstimation en cours...")

probit = sm.Probit(y, X_const)
result = probit.fit(disp=1, maxiter=100)

# ============================================================================
# RÉSULTATS
# ============================================================================
print(result.summary())

# ============================================================================
# EFFETS MARGINAUX (à la moyenne)
# ============================================================================
marginal_effects = result.get_margeff(at='mean')
print("\n" + "="*60)
print("EFFETS MARGINAUX (à la moyenne)")
print("="*60)
print(marginal_effects.summary())

# ============================================================================
# SAUVEGARDE
# ============================================================================
with open(path + 'probit_model1_summary.txt', 'w') as f:
    f.write(result.summary().as_text())
    f.write("\n\n" + "="*60 + "\n")
    f.write("EFFETS MARGINAUX\n")
    f.write("="*60 + "\n")
    f.write(marginal_effects.summary().as_text())

print(f"\n✓ Résultats sauvegardés : probit_model1_summary.txt")

Observations : 494,483
Transfrontaliers : 44,264 (8.95%)
Variables : 66 (dont constante)

Estimation en cours...
Optimization terminated successfully.
         Current function value: 0.229292
         Iterations 10
                          Probit Regression Results                           
Dep. Variable:        TRANSFRONTALIER   No. Observations:               494483
Model:                         Probit   Df Residuals:                   494417
Method:                           MLE   Df Model:                           65
Date:                Wed, 07 Jan 2026   Pseudo R-squ.:                  0.2393
Time:                        12:27:56   Log-Likelihood:            -1.1338e+05
converged:                       True   LL-Null:                   -1.4905e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                      coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------

# Modèle Probit : Déterminants du travail transfrontalier dans le Grand Est


Ce rapport présente l'analyse économétrique des déterminants individuels de la probabilité d'être travailleur transfrontalier dans la région Grand Est. L'estimation repose sur un modèle **Probit binaire** appliqué à **494 483 observations** issues du Recensement de la Population 2022 (INSEE) et de la base de donnée MOBPRO des mobilités professionnelles.

**Résultat principal** : La probabilité moyenne d'être transfrontalier est de **8,95%** dans la population étudiée. La localisation géographique (proximité frontalière) constitue le déterminant le plus puissant, suivie par la nationalité, le secteur d'activité et les caractéristiques socioprofessionnelles.

### Convention de lecture

Les résultats sont présentés sous deux formes complémentaires :

1. **Effet marginal (pp)** : variation de la probabilité en points de pourcentage. *Exemple : +5 pp signifie que la probabilité passe de 8,95% à 13,95%.*

2. **Odds Ratio (OR)** : facteur multiplicatif des "chances" d'être transfrontalier (lecture logit équivalente). *Exemple : OR = 2,0 signifie que les chances sont doublées.*

### Classification des variables selon le risque d'endogénéité

Les variables explicatives sont classées selon leur statut causal :

| Statut | Interprétation | Variables concernées |
|--------|----------------|----------------------|
| **Exogène** | Relation causale plausible | Département, âge, sexe, nationalité, lieu de naissance, diplôme, enfants, secteur, CSP |
| **Ambigu** | Causalité incertaine | Type de contrat, temps partiel |
| **Endogène** | Association statistique (causalité inverse possible) | Propriétaire, nombre de voitures, type de logement, superficie |

---

## 1. Effets géographiques : la proximité frontalière comme déterminant majeur

> **Variables exogènes** : Le département de résidence est généralement choisi avant l'emploi ou déterminé par des facteurs familiaux. L'interprétation causale est justifiée.

**Catégorie de référence** : Marne (51) — département intérieur sans frontière directe

| Département             | Effet marginal | Odds Ratio | Frontière principale |
|-------------            |:--------------:|:----------:|----------------------|
| Moselle (57)            | +15,35 pp***   | 4,71       | Luxembourg |
| Haut-Rhin (68)          | +13,17 pp***   | 3,87       | Suisse / Allemagne |
| Meurthe-et-Moselle (54) | +12,22 pp***   | 3,56       | Luxembourg |
| Meuse (55)              | +9,59 pp***    | 2,82       | Belgique / Luxembourg |
| Bas-Rhin (67)           | +9,27 pp***    | 2,73       | Allemagne |
| Vosges (88)             | +1,85 pp***    | 1,30       | (éloigné des frontières) |
| Aube (10)               | +0,66 pp       | 1,11       | Intérieur (NS) |
| Haute-Marne (52)        | +0,75 pp       | 1,12       | Intérieur (NS) |

*Note : pp = points de pourcentage ; OR = Odds Ratio ; NS = non significatif ; \*\*\* p < 0.001*

### Lecture des résultats

**En points de pourcentage :**
- Toutes choses égales par ailleurs, **résider en Moselle plutôt que dans la Marne augmente la probabilité d'être travailleur transfrontalier de 15,35 points de pourcentage**.
- **Résider dans le Haut-Rhin augmente la probabilité d'être transfrontalier de 13,17 points de pourcentage** par rapport à la Marne.
- **Résider en Meurthe-et-Moselle augmente la probabilité d'être transfrontalier de 12,22 points de pourcentage** par rapport à la Marne.

**En odds ratio :**
- **Les chances d'être transfrontalier sont multipliées par 4,7 pour un résident de Moselle** par rapport à un résident de la Marne.
- **Les chances sont multipliées par 3,9 pour un résident du Haut-Rhin** par rapport à la Marne.
- **Les chances sont multipliées par 3,6 pour un résident de Meurthe-et-Moselle** par rapport à la Marne.

### Interprétation économique

La hiérarchie des effets géographiques reflète fidèlement l'attractivité différenciée des marchés du travail frontaliers :

1. **Luxembourg** (Moselle, Meurthe-et-Moselle, Meuse) : salaires élevés, fiscalité avantageuse, forte demande de main-d'œuvre
2. **Suisse** (Haut-Rhin) : rémunérations très attractives dans l'industrie et les services
3. **Allemagne** (Bas-Rhin, Haut-Rhin) : proximité industrielle et complémentarité des marchés du travail

Les départements intérieurs (Aube, Haute-Marne) ne présentent pas de différence significative avec la Marne, confirmant que l'effet frontalier est le moteur principal.

---

## 2. Nationalité et origine géographique : l'effet des réseaux transnationaux

> **Variables exogènes** : La nationalité et le lieu de naissance sont déterminés avant l'entrée sur le marché du travail. L'interprétation causale est justifiée.

**Catégorie de référence** : Français né dans le Grand Est

| Variable                                | Effet marginal | Odds Ratio |
|----------                               |:--------------:|:----------:|
| Étranger (vs Français)                  | +3,12 pp***    | 1,56 |
| Né à l'étranger (vs né Grand Est)       | +2,20 pp***    | 1,40 |
| Né ailleurs en France (vs né Grand Est) | -0,19 pp**     | 0,97 |

### Lecture des résultats

**En points de pourcentage :**
- Toutes choses égales par ailleurs, **être de nationalité étrangère augmente la probabilité d'être transfrontalier de 3,12 points de pourcentage** par rapport aux Français.
- **Être né à l'étranger augmente la probabilité d'être transfrontalier de 2,20 points de pourcentage** par rapport aux personnes nées dans le Grand Est.
- **Être né dans une autre région française diminue la probabilité de 0,19 point de pourcentage**, suggérant un moindre ancrage dans les dynamiques frontalières locales.

**En odds ratio :**
- **Les chances d'être transfrontalier sont multipliées par 1,56 pour les étrangers** par rapport aux Français.
- **Les chances sont multipliées par 1,40 pour les personnes nées à l'étranger** par rapport à celles nées dans le Grand Est.

### Interprétation économique

Ces résultats mettent en évidence le rôle des **réseaux transnationaux** dans l'accès au travail frontalier :

- Les étrangers disposent souvent de **connexions familiales ou communautaires** dans les pays voisins facilitant l'information et l'embauche
- Les personnes nées à l'étranger possèdent fréquemment des **compétences linguistiques** (allemand, luxembourgeois, anglais) valorisées sur les marchés frontaliers
- Une **moindre aversion à la mobilité internationale** peut également jouer

---

## 3. Profil démographique

### 3.1 Effet de l'âge : une relation en U inversé

> **Variable exogène** : L'âge est une caractéristique individuelle fixe. L'interprétation causale est justifiée.

| Variable           | Effet marginal |
|----------          |:--------------:|
| Âge (linéaire)     | +0,21 pp par année*** |
| Âge² (quadratique) | -0,0025 pp par année²*** |

**Âge optimal calculé** : ≈ **41 ans**

### Lecture des résultats

- Toutes choses égales par ailleurs, **chaque année d'âge supplémentaire augmente la probabilité d'être transfrontalier de 0,21 point de pourcentage** jusqu'à environ 41 ans.
- **Au-delà de 41 ans, l'effet s'inverse** et la probabilité décroît progressivement.

### Interprétation économique

La relation en U inversé reflète deux dynamiques opposées :

- **Phase ascendante (< 41 ans)** : accumulation d'expérience professionnelle, stabilisation familiale permettant d'assumer les contraintes de mobilité, progression salariale rendant les trajets rentables
- **Phase descendante (> 41 ans)** : pénibilité croissante des trajets quotidiens, ancrage territorial renforcé, arbitrage en faveur de la qualité de vie

---

### 3.2 Effet du genre

> **Variable exogène** : Le sexe est une caractéristique individuelle fixe. L'interprétation causale est justifiée.

**Catégorie de référence** : Homme

| Variable | Effet marginal | Odds Ratio |
|----------|:--------------:|:----------:|
| Femme    | -0,79 pp***    | 0,88       |

### Lecture des résultats

- Toutes choses égales par ailleurs, **être une femme diminue la probabilité d'être transfrontalier de 0,79 point de pourcentage** par rapport aux hommes.
- **Les chances d'être transfrontalier sont réduites de 12% pour les femmes** (OR = 0,88) par rapport aux hommes.

### Interprétation économique

Ce différentiel de genre, cohérent avec la littérature sur les mobilités professionnelles, peut s'expliquer par :

1. **La charge familiale** : les femmes assurent encore majoritairement les responsabilités domestiques et parentales, peu compatibles avec les longs trajets
2. **La segmentation sectorielle** : surreprésentation féminine dans le secteur public et la santé, structurellement non-transfrontaliers

---

## 4. Situation familiale : la contrainte parentale

> **Variables exogènes** : Le nombre d'enfants et la structure familiale ne sont pas directement causés par le statut de travailleur transfrontalier. L'interprétation causale est justifiée.

**Catégorie de référence** : Sans enfant (NENFR = 0)

| Nombre d'enfants     | Effet marginal | Odds Ratio |
|------------------    |:--------------:|:----------:|
| 1 enfant             | -0,45 pp***     | 0,94 |
| 2 enfants            | -0,59 pp***     | 0,92 |
| 3 enfants            | -1,18 pp***     | 0,85 |
| 4 enfants ou plus    | -1,34 pp***     | 0,83 |
| Hors famille (isolé) | +0,52 pp***     | 1,07 |

| Statut conjugal      | Effet marginal | Odds Ratio |
|-----------------     |:--------------:|:----------:|
| Ne vit pas en couple | +0,14 pp*      | 1,02       |

*Note : \* p < 0.05*

### Lecture des résultats

- Toutes choses égales par ailleurs, **avoir un enfant diminue la probabilité d'être transfrontalier de 0,45 point de pourcentage** par rapport aux personnes sans enfant.
- **Avoir trois enfants diminue la probabilité d'être transfrontalier de 1,18 point de pourcentage** par rapport aux personnes sans enfant.
- **Avoir quatre enfants ou plus diminue la probabilité d'être transfrontalier de 1,34 point de pourcentage** par rapport aux personnes sans enfant.
- **Les personnes hors structure familiale voient leur probabilité augmenter de 0,52 point de pourcentage**.

### Interprétation économique

L'effet est **monotone décroissant** : chaque enfant supplémentaire réduit la probabilité de travail transfrontalier. Cette relation s'explique par :

- **L'incompatibilité temporelle** : trajets longs (souvent 1h-1h30 par trajet) difficilement conciliables avec les horaires scolaires et la garde d'enfants
- **Les coûts indirects** : garde d'enfants, activités extrascolaires nécessitant une présence parentale
- **L'arbitrage familial** : dans les couples, un des conjoints peut renoncer au travail frontalier pour assurer la logistique familiale

Le statut conjugal en lui-même (vivre en couple ou non) n'a qu'un effet marginal et faiblement significatif (p = 0.022), confirmant que c'est bien la **présence d'enfants**, et non le couple en soi, qui constitue la contrainte déterminante.

---

## 5. Caractéristiques professionnelles

### 5.1 Catégorie socioprofessionnelle

> **Variable exogène** : La CSP reflète une qualification et une trajectoire professionnelle préexistantes. L'interprétation causale est justifiée.

**Catégorie de référence** : Employés (GS_5)

| CSP                                       | Effet marginal | Odds Ratio |
|-----                                      |:--------------:|:----------:|
| Ouvriers                                  | +0,59 pp***    | 1,08 |
| Professions intermédiaires                | -0,25 pp***    | 0,97 |
| Cadres                                    | -0,53 pp***    | 0,93 |
| Artisans, commerçants, chefs d'entreprise | -1,93 pp***    | 0,77 |
| Agriculteurs                              | -0,68 pp       | 0,91 (NS) |

### Lecture des résultats

- Toutes choses égales par ailleurs, **être ouvrier augmente la probabilité d'être transfrontalier de 0,59 point de pourcentage** par rapport aux employés.
- **Être artisan, commerçant ou chef d'entreprise diminue la probabilité d'être transfrontalier de 1,93 point de pourcentage** par rapport aux employés.
- **Être cadre diminue la probabilité d'être transfrontalier de 0,53 point de pourcentage** par rapport aux employés.

### Interprétation économique

Les **ouvriers** constituent la catégorie la plus encline au travail transfrontalier, ce qui reflète :

- La **forte demande industrielle** au Luxembourg et en Allemagne (automobile, métallurgie, construction)
- Des **différentiels salariaux particulièrement attractifs** pour les qualifications ouvrières
- La **tradition industrielle transfrontalière** du bassin lorrain et alsacien

Les indépendants (artisans, commerçants) présentent logiquement la probabilité la plus faible, leur activité étant par nature **ancrée territorialement** (clientèle locale, établissement commercial).

---

### 5.2 Type de contrat et statut d'emploi

> **Variables ambiguës** : Le type de contrat peut être une condition d'accès au travail frontalier (exogène) ou une conséquence de l'emploi frontalier (endogène). L'interprétation reste prudente.

**Catégorie de référence** : CDI / Titulaire de la fonction publique (EMPL_16)

| Type de contrat       | Effet marginal | Odds Ratio |
|-----------------      |:--------------:|:----------:|
| Stagiaires rémunérés  | +2,61 pp***    | 1,49 |
| Intérim               | -0,49 pp***    | 0,93 |
| CDD, contrats courts  | -0,63 pp***    | 0,91 |
| Employeurs            | -2,98 pp***    | 0,69 |
| Apprentissage         | -3,33 pp***    | 0,64 |
| Indépendants          | -3,58 pp***    | 0,62 |
| Emplois aidés         | -4,70 pp***    | 0,52 |
| Aides familiaux       | -1,40 pp       | 0,82 (NS, p=0.069) |

### Lecture des résultats

- Les personnes en CDI ou titulaires de la fonction publique constituent la **norme du travail transfrontalier**.
- **Être stagiaire rémunéré est associé à une probabilité supérieure de 2,61 points de pourcentage**, probablement en lien avec des stages dans des entreprises luxembourgeoises ou allemandes.
- **Être en emploi aidé est associé à une probabilité inférieure de 4,70 points de pourcentage** par rapport aux CDI.
- **Être indépendant est associé à une probabilité inférieure de 3,58 points de pourcentage** par rapport aux CDI.
- **Être en apprentissage est associé à une probabilité inférieure de 3,33 points de pourcentage** par rapport aux CDI.

### Interprétation économique

Le travail transfrontalier est fortement associé à la **stabilité contractuelle** (CDI) pour plusieurs raisons :

- **Amortissement des coûts fixes** : l'investissement initial (trajet, éventuellement déménagement, adaptation) n'est rentable que sur le long terme
- **Accès au crédit** : les revenus frontaliers (souvent élevés) permettent l'accession à la propriété, conditionnée à un emploi stable
- **Sélection par les employeurs** : les entreprises luxembourgeoises et allemandes privilégient les CDI

Les formes précaires (emplois aidés, apprentissage, intérim) sont sous-représentées car elles concernent des populations éloignées des prérequis du travail frontalier (mobilité, qualification, stabilité).

---

### 5.3 Secteur d'activité

> **Variable exogène** : Le secteur d'activité détermine structurellement la possibilité d'être transfrontalier. L'interprétation causale est justifiée.

**Catégorie de référence** : Commerce, transports et services divers (NA5_GU)

| Secteur                             | Effet marginal | Odds Ratio |
|---------                            |:--------------:|:----------:|
| Industrie                           | +0,08 pp       | 1,01 (NS) |
| Construction                        | -0,13 pp       | 0,98 (NS, p=0.08) |
| Administration, enseignement, santé | -4,16 pp***    | 0,55 |
| Agriculture                         | -6,01 pp***    | 0,44 |

### Lecture des résultats

- Toutes choses égales par ailleurs, **travailler dans l'administration publique, l'enseignement ou la santé diminue la probabilité d'être transfrontalier de 4,16 points de pourcentage** par rapport au secteur tertiaire marchand.
- **Travailler dans l'agriculture diminue la probabilité d'être transfrontalier de 6,01 points de pourcentage** par rapport au secteur tertiaire marchand.
- **Les chances d'être transfrontalier sont réduites de 45% dans le secteur public** (OR = 0,55) et de **56% dans l'agriculture** (OR = 0,44).
- **L'industrie et la construction ne diffèrent pas significativement** du secteur tertiaire marchand.

### Interprétation économique

Le secteur public (administration, enseignement, santé) est **structurellement non-transfrontalier** :

- Ces emplois sont liés au **territoire national** (fonctionnaires, hospitaliers, enseignants)
- Les systèmes de santé et d'éducation ne sont pas intégrés entre pays
- Les concours et recrutements sont nationaux

L'agriculture, par nature **ancrée géographiquement** (exploitation, terres), exclut de facto la mobilité transfrontalière.

---

## 6. Capital humain : une relation non linéaire avec le diplôme

> **Variable exogène** : Le diplôme est obtenu avant l'entrée sur le marché du travail. L'interprétation causale est justifiée.

**Catégorie de référence** : CAP, BEP ou diplôme équivalent (DIPL_13)

| Niveau de diplôme                  | Effet marginal | Odds Ratio |
|-------------------                 |:--------------:|:----------:|
| Doctorat de recherche              | +1,54 pp***    | 1,21 |
| Master, Bac+5, Grande école        | +1,40 pp***    | 1,19 |
| CEP (certificat d'études primaires)| +0,77 pp***    | 1,10 |
| Licence, Bac+3/4                   | +0,50 pp***    | 1,07 |
| BEPC, Brevet                       | -0,14 pp       | 0,98 (NS) 
| Bac général/technologique          | -0,31 pp***    | 0,96 |
| Bac professionnel                  | -0,22 pp**     | 0,97 |
| BTS, DUT, Bac+2                    | -0,31 pp***    | 0,96 |
| Aucun diplôme (scolarité collège)  | -0,61 pp***    | 0,92 |
| Aucun diplôme (scolarité primaire) | -1,60 pp***    | 0,80 |
| Pas de scolarité                   | -3,28 pp***    | 0,62 |

### Lecture des résultats

- Toutes choses égales par ailleurs, **être titulaire d'un doctorat augmente la probabilité d'être transfrontalier de 1,54 point de pourcentage** par rapport aux titulaires d'un CAP/BEP.
- **Être titulaire d'un Master ou diplôme Bac+5 augmente la probabilité d'être transfrontalier de 1,40 point de pourcentage** par rapport aux titulaires d'un CAP/BEP.
- **Être titulaire du CEP augmente la probabilité d'être transfrontalier de 0,77 point de pourcentage** par rapport aux titulaires d'un CAP/BEP.
- **N'avoir aucune scolarité diminue la probabilité d'être transfrontalier de 3,28 points de pourcentage** par rapport aux titulaires d'un CAP/BEP.

### Interprétation économique

La relation entre diplôme et travail transfrontalier est **non linéaire**, avec deux profils distincts :

**Profil 1 — Très diplômés (Master+)** :
- Accès à des postes qualifiés au Luxembourg (finance, audit, consulting, institutions européennes)
- Maîtrise des langues étrangères
- Mobilité internationale facilitée

**Profil 2 — Faiblement qualifiés avec expérience (CEP)** :
- Main-d'œuvre ouvrière expérimentée, recherchée dans l'industrie et la construction frontalières
- Différentiel salarial très attractif pour ces niveaux de qualification

**Profil intermédiaire (Bac, BTS) moins mobile** :
- Qualification trouvant des débouchés locaux satisfaisants
- Moindre incitation relative à supporter les coûts de la mobilité

---

## 7. Conditions de logement et motorisation

> **Variables endogènes** : Les variables de cette section présentent un **risque élevé de causalité inverse**. Les revenus plus élevés des travailleurs transfrontaliers peuvent permettre d'accéder à la propriété, d'habiter en maison individuelle ou de posséder plusieurs véhicules. Les résultats ci-dessous doivent être interprétés comme des **associations statistiques** et non comme des relations causales.

---

### 7.1 Motorisation du ménage

**Catégorie de référence** : 1 voiture

| Nombre de voitures | Effet marginal | Odds Ratio |
|--------------------|:--------------:|:----------:|
| 3 voitures ou plus | +1,05 pp***    | 1,14 |
| 2 voitures         | +0,74 pp***    | 1,10 |
| Aucune voiture     | -2,04 pp***    | 0,76 |

### Lecture des résultats (association)

- **Les ménages sans voiture sont associés à une probabilité inférieure de 2,04 points de pourcentage** d'avoir un membre transfrontalier par rapport aux ménages avec une voiture.
- **Les ménages possédant 2 voitures sont associés à une probabilité supérieure de 0,74 point de pourcentage**.
- **Les ménages possédant 3 voitures ou plus sont associés à une probabilité supérieure de 1,05 point de pourcentage**.

### Interprétation économique

L'automobile apparaît **indispensable** au travail transfrontalier dans le Grand Est :

- Les transports en commun transfrontaliers restent **peu développés** (à l'exception de quelques lignes vers le Luxembourg)
- Les trajets domicile-travail sont souvent longs (30 à 80 km)
- Les horaires décalés (postes, industrie) sont incompatibles avec les transports collectifs

**⚠️ Réserve méthodologique** : La multi-motorisation peut également être une **conséquence** des revenus frontaliers élevés. Sans variable instrumentale, il est impossible de distinguer la cause de l'effet.

---

### 7.2 Statut d'occupation du logement

**Catégorie de référence** : Propriétaire

| Statut d'occupation    | Effet marginal | Odds Ratio |
|---------------------   |:--------------:|:----------:|
| Locataire non-HLM      | -0,51 pp***    | 0,93 |
| Locataire meublé/hôtel | -1,31 pp***    | 0,83 |
| Logé gratuitement      | -1,79 pp***    | 0,78 |
| Locataire HLM          | -3,11 pp***    | 0,64 |

### Lecture des résultats (association)

- **Les propriétaires sont plus fréquemment associés au statut de travailleur transfrontalier** que toutes les autres catégories.
- **Les locataires en HLM sont associés à une probabilité inférieure de 3,11 points de pourcentage** par rapport aux propriétaires.
- **Les personnes logées gratuitement sont associées à une probabilité inférieure de 1,79 point de pourcentage** par rapport aux propriétaires.

### Interprétation économique

Cette association peut refléter plusieurs mécanismes :

1. **Effet richesse (causalité inverse probable)** : les revenus frontaliers élevés permettent l'accession à la propriété
2. **Ancrage résidentiel** : les propriétaires, plus stables géographiquement, peuvent optimiser leur localisation par rapport aux frontières
3. **Sélection sociale** : les locataires HLM, souvent en situation de précarité économique, sont éloignés des prérequis du travail frontalier (mobilité, qualification, réseau)

**Réserve méthodologique** : La propriété est très probablement une **conséquence** du travail frontalier (revenus élevés → accession). Un modèle à équations simultanées serait nécessaire pour identifier l'effet causal.

---

### 7.3 Type de logement

**Catégorie de référence** : Maison individuelle

| Type de logement      | Effet marginal | Odds Ratio |
|------------------     |:--------------:|:----------:|
| Appartement           | -1,04 pp***    | 0,87 |
| Logement-foyer        | -3,01 pp***    | 0,64 |
| Chambre d'hôtel       | -3,89 pp***    | 0,56 |
| Pièce indépendante    | -3,98 pp***    | 0,55 |
| Habitation de fortune | -1,11 pp       | 0,86 (NS) |

### Lecture des résultats (association)

- **Les personnes vivant en maison individuelle sont plus fréquemment transfrontalières** que celles vivant en appartement ou en logement collectif.
- **Vivre en appartement est associé à une probabilité inférieure de 1,04 point de pourcentage** par rapport à la maison.
- **Vivre en logement-foyer est associé à une probabilité inférieure de 3,01 points de pourcentage** par rapport à la maison.

### Interprétation économique

L'association positive avec la maison individuelle peut s'expliquer par :

1. **Localisation géographique** : les maisons individuelles sont plus fréquentes en zones périurbaines et rurales, souvent plus proches des frontières que les centres urbains
2. **Causalité inverse** : les revenus frontaliers permettent d'accéder à la maison individuelle
3. **Profil sociologique** : les ménages en maison sont souvent des couples avec enfants et deux actifs, profil compatible avec le travail frontalier

---

## 8. Temps de travail

> **Variable ambiguë** : Le temps partiel peut être une cause (contrainte de mobilité) ou une conséquence (choix permis par des revenus élevés) du travail transfrontalier.

**Catégorie de référence** : Temps complet

| Variable      | Effet marginal | Odds Ratio |
|----------     |:--------------:|:----------:|
| Temps partiel | +0,77 pp***    | 1,10 |

### Lecture des résultats

- **Travailler à temps partiel est associé à une probabilité supérieure de 0,77 point de pourcentage** par rapport au temps complet.

### Interprétation économique

Ce résultat, contre-intuitif au premier abord, peut s'expliquer par :

1. **Temps partiels choisis** : certains travailleurs frontaliers optent pour un temps partiel afin de concilier trajets longs et vie familiale
2. **Organisation du travail au Luxembourg** : certains emplois frontaliers (notamment dans les services) sont structurés en temps partiel
3. **Effet de composition** : le temps partiel peut refléter des arrangements familiaux où un conjoint travaille à temps partiel pendant que l'autre effectue les trajets frontaliers

** Réserve méthodologique** : Cette variable peut refléter un **arbitrage endogène** entre revenus et qualité de vie.

---

## 9. Inscription dans un établissement d'enseignement

>  **Variable exogène** : Le statut d'étudiant est une caractéristique individuelle préexistante. L'interprétation causale est justifiée.

**Catégorie de référence** : Non inscrit dans un établissement d'enseignement

| Variable | Effet marginal | Odds Ratio |
|----------|:--------------:|:----------:|
| Étudiant | -2,08 pp***    | 0,76 |

### Lecture des résultats

- Toutes choses égales par ailleurs, **être inscrit dans un établissement d'enseignement diminue la probabilité d'être transfrontalier de 2,08 points de pourcentage**.
- **Les chances d'être transfrontalier sont réduites de 24% pour les étudiants** (OR = 0,76).

### Interprétation économique

Les étudiants, même en emploi, sont moins susceptibles d'être transfrontaliers car :

- Leurs emplois sont souvent **temporaires et locaux** (jobs étudiants)
- La **contrainte de présence** aux cours limite la mobilité géographique
- Les **stages frontaliers** sont captés par la variable EMPL_14 (stagiaires rémunérés, effet positif)

---

## 10. Synthèse des déterminants

### 10.1 Facteurs favorisant le travail transfrontalier (variables exogènes)

| Rang | Déterminant                | Effet marginal | Odds Ratio | Interprétation |
|:----:|-------------               |:--------------:|:----------:|----------------|
| 1 | Résider en Moselle            | +15,35 pp      | 4,71 | Proximité Luxembourg |
| 2 | Résider dans le Haut-Rhin     | +13,17 pp      | 3,87 | Proximité Suisse/Allemagne |
| 3 | Résider en Meurthe-et-Moselle | +12,22 pp      | 3,56 | Proximité Luxembourg |
| 4 | Être de nationalité étrangère | +3,12 pp       | 1,56 | Réseaux transnationaux |
| 5 | Être né à l'étranger          | +2,20 pp       | 1,40 | Compétences linguistiques |
| 6 | Doctorat ou Master            | +1,4 à +1,5 pp | 1,19-1,21 | Postes qualifiés |
| 7 | Être ouvrier                  | +0,59 pp       | 1,08 | Demande industrielle |
| 8 | Âge ~40 ans                   | Optimum |   —  | Expérience + mobilité |

### 10.2 Facteurs freinant le travail transfrontalier (variables exogènes)

| Rang | Déterminant                | Effet marginal | Odds Ratio | Interprétation |
|:----:|-------------               |:--------------:|:----------:|----------------|
| 1 | Travailler dans l'agriculture | -6,01 pp       | 0,44 | Ancrage territorial |
| 2 | Secteur public                | -4,16 pp       | 0,55 | Emplois nationaux |
| 3 | Pas de scolarité              | -3,28 pp       | 0,62 | Barrières à l'emploi |
| 4 | Être étudiant                 | -2,08 pp       | 0,76 | Emplois temporaires |
| 5 | Artisan, commerçant           | -1,93 pp       | 0,77 | Clientèle locale |
| 6 | 4+ enfants                    | -1,34 pp       | 0,83 | Contrainte familiale |
| 7 | Être femme                    | -0,79 pp       | 0,88 | Charge familiale |

### 10.3 Variables associées (endogènes ou ambiguës — interprétation prudente)

| Variable            | Effet marginal | Odds Ratio | Risque d'endogénéité |
|----------           |:--------------:|:----------:|----------------------|
| Aucune voiture      | -2,04 pp       | 0,76 | 🔴 Élevé |
| Locataire HLM       | -3,11 pp       | 0,64 | 🔴 Élevé |
| Propriétaire (réf.) | —              | —    | 🔴 Élevé |
| Maison (réf.)       | —              | —    | 🔴 Élevé |
| Temps partiel       | +0,77 pp       | 1,10 | 🟡 Modéré |
| CDI (réf.)          | —              | —    | 🟡 Modéré |

---

## 11. Qualité de l'estimation économétrique

### 11.1 Indicateurs de qualité globale

| Indicateur | Valeur | Interprétation |
|------------|:------:|----------------|
| **Nombre d'observations** | 494 483 | Échantillon très large, puissance statistique élevée |
| **Pseudo R² (McFadden)** | 0,2393 | Bon ajustement pour un modèle binaire sur données individuelles |
| **Log-vraisemblance** | -113 380 | — |
| **Log-vraisemblance nulle** | -149 050 | — |
| **Test LR (χ²)** | 71 340 | — |
| **p-value LR** | < 0,001 | Modèle globalement très significatif |
| **Convergence** | Oui (10 itérations) | Algorithme stable |

### 11.2 Interprétation du Pseudo R²

Un Pseudo R² de McFadden de **0,24** est considéré comme **bon à très bon** pour un modèle de choix discret sur données individuelles. À titre de comparaison :

- Un Pseudo R² entre 0,20 et 0,40 est généralement jugé satisfaisant à excellent en économétrie des choix
- Un Pseudo R² > 0,40 serait exceptionnel et pourrait même suggérer un sur-ajustement

Le modèle capture donc une **part substantielle de la variance** du phénomène étudié.

### 11.3 Significativité des variables

| Catégorie                             | Nombre |
|-----------                            |:------:|
| Variables significatives au seuil 1%  | 56 / 65|
| Variables significatives au seuil 5%  | 2 / 65 |
| Variables significatives au seuil 10% | 2 / 65 |
| Variables non significatives          | 5 / 65 |

**Variables non significatives** : GS_1 (agriculteurs exploitants), DEPT_10 (Aube), DEPT_52 (Haute-Marne), NA5_BE (industrie), TYPL_5 (habitation de fortune).

Ces non-significativités sont cohérentes économiquement (effectifs faibles ou absence de différence réelle avec la référence).

---

## 12. Limites méthodologiques et extensions futures

### 12.1 Limites du modèle actuel

#### Endogénéité de certaines variables

Plusieurs variables explicatives (propriété, motorisation, type de logement) sont potentiellement **endogènes** : elles peuvent être des conséquences plutôt que des causes du travail transfrontalier. Sans correction, les coefficients estimés sont **biaisés** et ne permettent pas d'inférence causale.

#### Variables omises

Le modèle n'inclut pas :
- La **distance précise à la frontière** (seul le département est disponible)
- Les **compétences linguistiques** (allemand, luxembourgeois)
- Le **différentiel salarial individuel** entre emploi local et frontalier
- Les **préférences individuelles** (aversion aux trajets, attachement territorial)

#### Hétérogénéité non observée

Des caractéristiques individuelles non mesurées (motivation, réseau professionnel, santé) peuvent influencer simultanément le choix de résidence et le travail frontalier.

#### Coupe transversale

L'analyse sur une seule année (2022) ne permet pas d'étudier les dynamiques temporelles (entrées/sorties du travail frontalier, effets de la conjoncture).

---

### 12.2 Extensions futures recommandées

#### 1. Traitement de l'endogénéité

| Méthode | Principe | Application possible |
|---------|----------|----------------------|
| **Probit à Variables Instrumentales (IV Probit)** | Trouver un instrument Z affectant la variable endogène mais pas Y directement | Instrument potentiel : taux de propriétaires dans la commune d'origine |
| **Control Function Approach** | Inclure les résidus de la 1ère étape dans le probit | Permet de tester l'endogénéité |
| **Probit bivarié récursif** | Modéliser simultanément P(propriétaire) et P(transfrontalier) | Identification par restrictions d'exclusion |

#### 2. Modèle Probit avec interactions par genre (prévu)

Objectif : Tester si les déterminants ont des **effets différenciés** selon le sexe.

Spécification :
$$P(Y_i = 1) = \Phi(\beta_0 + \beta_1 Femme_i + \beta_2 X_i + \beta_3 (Femme_i \times X_i))$$

Permettra de répondre à : *L'effet des enfants est-il plus fort pour les femmes ?*

#### 3. Modèle Probit bivarié pour les couples (prévu)

Objectif : Tester la **coordination des décisions** au sein des couples.

Spécification :
$$Y_{1i}^* = X_{1i}'\beta_1 + \varepsilon_{1i}$$
$$Y_{2i}^* = X_{2i}'\beta_2 + \varepsilon_{2i}$$

avec $\text{Corr}(\varepsilon_1, \varepsilon_2) = \rho$

Permettra de répondre à : *Les choix professionnels des conjoints sont-ils coordonnés ?*


#### 5. Extensions de données

- **Panel** : Suivre les individus sur plusieurs années pour identifier les transitions vers le travail frontalier
- **Variables géographiques fines** : Distance à la frontière, accessibilité routière
- **Variables fiscales** : Revenus individuels pour calculer le différentiel salarial

---

## 13. Conclusion

Cette analyse économétrique met en évidence une **hiérarchie claire des déterminants** du travail transfrontalier dans le Grand Est :

1. **La géographie domine** : résider dans un département frontalier (Moselle, Haut-Rhin, Meurthe-et-Moselle) est le facteur le plus déterminant, avec des effets marginaux de 12 à 15 points de pourcentage et des odds ratios de 3,5 à 4,7.

2. **Les réseaux transnationaux comptent** : la nationalité étrangère et le lieu de naissance à l'étranger facilitent l'accès au travail frontalier, probablement via les compétences linguistiques et les connexions sociales.

3. **Le profil socioprofessionnel sélectionne** : le travail frontalier concerne prioritairement les ouvriers en CDI et les cadres très diplômés, excluant les indépendants, le secteur public et les formes d'emploi précaires.

4. **La famille contraint** : la présence d'enfants réduit significativement la probabilité d'être transfrontalier, reflétant l'incompatibilité entre trajets longs et contraintes parentales.

5. **Les variables de logement et motorisation sont associées** au travail frontalier, mais le sens de la causalité reste incertain et nécessite des investigations complémentaires.

Ces résultats fournissent une base solide pour l'estimation des modèles complémentaires prévus (probit avec interactions par genre, probit bivarié pour les décisions de couple) et pour l'élaboration de recommandations en matière de politique publique (transport, garde d'enfants, formation linguistique).

---

*Rapport généré le 7 janvier 2026 — Projet INSEE Mobilité Transfrontalière Grand Est
*Romain Mehdi FEHRI
*Master 2 Statistique, Économétrie & Data Science — Université de Strasbourg*